Import UK data

Healed data are missing - Actual cases can't be calculated

In [2]:
import pandas as pd
import glob
import os

#get path for os 
path = os.getcwd() + "/data_source/UK/covid-19-uk-data-master/data/covid-19-indicators-uk.csv"
path = path.replace("/",os.path.sep)

columns = ["total_cases","daily_cases","total_dead","daily_dead", "total_test","daily_test"]

dataset = pd.read_csv(path, index_col = 0)

Grouping by data and indicator to sum the confirmed cases, deaths and tests among different regions

In [3]:
df = df[df. == "UK"]
dataset = dataset.groupby(["Date","Indicator"]).sum()

Creating a new dataframe with our specific format

In [8]:
uk_dataset = pd.DataFrame(index = dataset.index.levels[0] , columns = columns)
uk_dataset.reset_index()
uk_dataset = uk_dataset.fillna(0)

dataset

Value
Date       Indicator             
2020-01-24 ConfirmedCases       0
           Deaths               0
           Tests                2
2020-01-25 ConfirmedCases       0
           Deaths               0
...                           ...
2020-04-28 Deaths           50788
           Tests           680584
2020-04-29 ConfirmedCases  189347
           Deaths           52308
           Tests           716238

[291 rows x 1 columns]

Filling the new dataframe with the avaiable data

In [5]:
from datetime import datetime, timedelta
day = []
for days in dataset.index.levels[0]:
    try:
        #from 2nd row ---- to subtract a day from the date we had to convert it to a date and then back to a string
        yesterday = (datetime.strptime(days,"%Y-%m-%d")-timedelta(days=1)).strftime("%Y-%m-%d")
        uk_dataset.loc[days]["daily_cases"] = dataset.loc[days,"ConfirmedCases"]["Value"] - dataset.loc[yesterday,"ConfirmedCases"]["Value"]
        uk_dataset.loc[days]["daily_dead"] = dataset.loc[days,"Deaths"]["Value"] - dataset.loc[yesterday,"Deaths"]["Value"]
        uk_dataset.loc[days]["daily_test"] = dataset.loc[days,"Tests"]["Value"] - dataset.loc[yesterday,"Tests"]["Value"]
    except:
        #First row
        uk_dataset.loc[days]["daily_cases"] = dataset.loc[days,"ConfirmedCases"]["Value"]
        uk_dataset.loc[days]["daily_dead"] = dataset.loc[days,"Deaths"]["Value"]
        uk_dataset.loc[days]["daily_test"] = dataset.loc[days,"Tests"]["Value"]
    finally:
        #Single row
        uk_dataset.loc[days]["total_cases"] = dataset.loc[days,"ConfirmedCases"]["Value"]
        #Missing recoveres -> uk_dataset.loc[days]["actual_cases"] =
        uk_dataset.loc[days]["total_dead"] = dataset.loc[days,"Deaths"]["Value"]
        uk_dataset.loc[days]["total_test"] = dataset.loc[days,"Tests"]["Value"]



Converting dates from String to Datetime

In [6]:
import datetime as dt

index_list = uk_dataset.index.tolist()

for i in range(len(index_list)):
    index_list[i] = dt.datetime.strptime(index_list[i], '%Y-%m-%d').date()

uk_dataset.index = index_list
uk_dataset.index.name = 'date'

uk_dataset.tail()

,total_cases,daily_cases,total_dead,daily_dead,total_test,daily_test
date,,,,,,
2020-04-25,284934,7264,47430,1675,611243,77162
2020-04-26,290870,5936,48231,801,639675,28432
2020-04-27,296129,5259,48912,681,668730,29055
2020-04-28,300645,4516,50788,1876,680584,11854
2020-04-29,189347,-111298,52308,1520,716238,35654


In [6]:
#get path for os 
path = os.getcwd() + "/parsed_data/united_kingdom/"
path = path.replace("/",os.path.sep)
#save data as CSV 
uk_dataset.to_csv (path + "united_kingdom.csv", index = True, header=True)